# June 2020

In [1]:
Sequence data from Ensembl (https://useast.ensembl.org/info/data/ftp/index.html).

Filename = Homo_sapiens.GRCh38.ncrna.fa.gz
Num sequences = 58028
Shortest sequence: ENSG00000056678.11 ENST00000411692.1 lncRNA CGGCC
Top 5 longest sequence lengths: {37027, 37852, 49287, 91667, 205012}

SyntaxError: invalid syntax (<ipython-input-1-e5c1993847e6>, line 1)

In [2]:
import pandas as pd
infile='lncRNA.HeLa.features.csv'
scored_sequence = pd.read_csv(infile,header=0)
scored_sequence.head()

,label,seqname,AAAA,AAAC,AAAG,AAAT,AACA,AACC,AACG,AACT,...,TTCG,TTCT,TTGA,TTGC,TTGG,TTGT,TTTA,TTTC,TTTG,TTTT
0,-2.037350,ENSG00000259891.1,71,8,9,16,16,5,0,1,...,2,9,20,11,8,8,11,3,19,11
1,-0.484711,ENSG00000215808.4,133,46,53,63,48,35,2,29,...,4,63,51,36,32,50,55,53,75,128
2,-1.929540,ENSG00000236088.10,187,76,90,105,85,50,6,46,...,8,138,84,63,56,96,125,128,127,280
3,-2.027480,ENSG00000272202.1,194,80,97,113,87,52,8,49,...,9,139,85,64,58,96,125,131,127,280
4,-4.424920,ENSG00000180422.3,196,82,99,117,92,54,9,50,...,12,149,89,72,66,100,127,141,136,289


In [3]:
scored_sequence.shape

(1152, 258)

In [4]:
def split_train_test(dataset,train_portion):    
    # Use pandas sample() to randomize the order (i.e. random sample without replacement).
    # Side note. Data frame can be randomized while keeping header in place.
    # This is repeatable with 42.
    middle = int(len(dataset)*train_portion)
    random = dataset.sample(frac=1,random_state=42)
    train_set = random[:middle]
    test_set = random[middle:]
    return train_set,test_set
TRAIN_PORTION=0.8
raw_train, raw_test = split_train_test(scored_sequence,TRAIN_PORTION)
shuffle_train = raw_train.sample(frac=1,random_state=17)
y_train = shuffle_train['label']
X_train_ID = shuffle_train['seqname'] # Save for later?
X_train = shuffle_train.drop(['label','seqname'],axis=1)
X_train.shape
y_train.shape

(921,)

In [5]:
# Normalize by row sum. 
X_norm=X_train.div(X_train.sum(axis=1), axis=0)
# After normalization, each row should sum to 1.
X_norm.sum(axis=1)

263    1.0
571    1.0
250    1.0
462    1.0
887    1.0
      ... 
572    1.0
259    1.0
654    1.0
243    1.0
74     1.0
Length: 921, dtype: float64

In [6]:
X_norm[:5]

,AAAA,AAAC,AAAG,AAAT,AACA,AACC,AACG,AACT,AAGA,AAGC,...,TTCG,TTCT,TTGA,TTGC,TTGG,TTGT,TTTA,TTTC,TTTG,TTTT
263,0.010881,0.004722,0.006008,0.006880,0.004896,0.003324,0.000977,0.003841,0.005755,0.004122,...,0.000934,0.006197,0.004095,0.003631,0.004686,0.004674,0.005617,0.006076,0.006021,0.012000
571,0.010691,0.004694,0.005955,0.006850,0.004885,0.003336,0.000920,0.003905,0.005843,0.003993,...,0.000856,0.006291,0.004115,0.003674,0.004618,0.004665,0.005621,0.006096,0.006070,0.011480
250,0.010939,0.004759,0.006022,0.006885,0.004925,0.003325,0.000974,0.003871,0.005788,0.004120,...,0.000922,0.006168,0.004069,0.003630,0.004659,0.004670,0.005608,0.006082,0.006029,0.012085
462,0.010985,0.004760,0.006003,0.007005,0.004977,0.003386,0.000953,0.003918,0.005859,0.004024,...,0.000873,0.006300,0.004175,0.003700,0.004671,0.004743,0.005725,0.006108,0.006128,0.011548
887,0.010716,0.004637,0.006001,0.006978,0.004885,0.003306,0.000906,0.003936,0.005823,0.003999,...,0.000879,0.006478,0.004219,0.003680,0.004627,0.004846,0.005824,0.006209,0.006229,0.011662


In [11]:
y_train.shape, X_norm.shape

((921,), (921, 256))

In [15]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
lin = LinearRegression()
lin.fit(X_norm,y_train)
y_pred=lin.score(X_norm,y_train)
from sklearn.metrics import mean_squared_error
y_pred
#mse=mean_squared_error(y_train,y_pred)
#mse

0.3049687650485403